## True AlexNet implementation with generator and RGB images
Each image 320x240x3

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11190910675724739468]

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
width, height = 320, 240
train_data_dir = '../gen_x3/train'
test_data_dir = '../gen_x3/test'
nb_train_samples = 9446
nb_test_samples = 2362
epochs = 25
batch_size = 4

input_shape = (width, height, 3)

In [21]:
model = Sequential()
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
# summarize model and apply checkpoints
print(model.summary())
filepath = 'generator_True_AlexNet-{epoch:02d}-{loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 78, 58, 96)        34944     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 39, 29, 96)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 35, 25, 256)       614656    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 17, 12, 256)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 10, 384)       885120    
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 8, 384)        1327488   
_________________________________________________________________
dropout_22 (Dropout)         (None, 13, 8, 384)        0         
__________

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.20,
    zoom_range=0.15,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(width, height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(width, height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

Found 9446 images belonging to 6 classes.
Found 2362 images belonging to 6 classes.


In [9]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

Epoch 1/25
   2/2361 [..............................] - ETA: 6:35:53 - loss: 5.9735 - acc: 0.1250    

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())
print("Accuracy: %.2f%%" % (score[1]*100))
print('Test loss:', score[0])
print('Test accuracy', score[1])

In [ ]:
#summarize for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#summarize for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_json = model.to_json()
with open('generator_True_AlexNet_model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('generator_True_AlexNet_weights.h5')
print('Saved model weights')